In [42]:
import pandas as pd

file_path = '../../data/school_health_preprocessed.csv'

try:
    data = pd.read_csv(file_path, encoding='utf-8')
    print("데이터 로드 성공!")
except FileNotFoundError:
    print("파일을 찾을 수 없습니다. 경로를 확인하세요.")


print(data[['하루30분이상운동']].isnull().sum())

cleaned_data = data.dropna(subset=['BMI', '하루30분이상운동'])
print(f"Cleaned data size: {cleaned_data.shape}")

# 성별로 데이터 분리
male_data = cleaned_data[cleaned_data['성별'] == '남']
female_data = cleaned_data[cleaned_data['성별'] == '여']



데이터 로드 성공!
하루30분이상운동    35081
dtype: int64
Cleaned data size: (49790, 111)


In [43]:
import pandas as pd

# 필요한 열 추출
df = data[['BMI', '주3회이상운동', '학교급', '성별', '하루30분이상운동']]

# '주3회이상운동' 관련 데이터 결측치 제거
cleaned_data_exercise_3times = df.dropna(subset=['BMI', '주3회이상운동', '학교급', '성별'])
print(f"Cleaned data size for 주3회이상운동: {cleaned_data_exercise_3times.shape}")
print(cleaned_data_exercise_3times.head())

# '주3회이상운동' 매핑 : 초등학생 대상
exercise_mapping_3times = {
    1.0: '예',
    2.0: '아니오'
}

cleaned_data_exercise_3times['주 3회 운동여부'] = cleaned_data_exercise_3times['주3회이상운동'].map(exercise_mapping_3times)
print("\n[주3회이상운동 매핑 결과 (초등학생 대상)]")
print(cleaned_data_exercise_3times[['BMI', '주3회이상운동', '주 3회 운동여부','성별','학교급']].head())

# '하루30분이상운동' 관련 데이터 결측치 제거
cleaned_data_exercise_30min = df.dropna(subset=['BMI', '하루30분이상운동', '학교급', '성별'])
print(f"Cleaned data size for 하루30분이상운동: {cleaned_data_exercise_30min.shape}")
print(cleaned_data_exercise_30min.head())

# '하루30분이상운동' 매핑 : 중, 고등학생 대상
exercise_mapping_30min = {
    1.0: '거의 안함',
    2.0: '주에 1~2회',
    3.0: '주에 3~4회',
    4.0: '주에 5회 이상'
}

cleaned_data_exercise_30min['운동빈도'] = cleaned_data_exercise_30min['하루30분이상운동'].map(exercise_mapping_30min)
print("\n[하루30분이상운동 매핑 결과 (중, 고등학생 대상)]")
print(cleaned_data_exercise_30min[['BMI', '하루30분이상운동', '운동빈도','성별','학교급']].head())


Cleaned data size for 주3회이상운동: (32715, 5)
         BMI  주3회이상운동 학교급 성별  하루30분이상운동
0  15.667728      2.0   초  여        NaN
1  15.644444      2.0   초  여        NaN
2  15.159148      2.0   초  여        NaN
3  22.701628      2.0   초  여        NaN
4  14.870556      1.0   초  여        NaN

[주3회이상운동 매핑 결과 (초등학생 대상)]
         BMI  주3회이상운동 주 3회 운동여부 성별 학교급
0  15.667728      2.0       아니오  여   초
1  15.644444      2.0       아니오  여   초
2  15.159148      2.0       아니오  여   초
3  22.701628      2.0       아니오  여   초
4  14.870556      1.0         예  여   초
Cleaned data size for 하루30분이상운동: (49790, 5)
             BMI  주3회이상운동 학교급 성별  하루30분이상운동
29623  15.238947      NaN   중  남        3.0
29624  20.342891      NaN   중  여        1.0
29625  24.689836      NaN   중  남        3.0
29626  20.877623      NaN   중  여        2.0
29627  17.201060      NaN   중  여        1.0

[하루30분이상운동 매핑 결과 (중, 고등학생 대상)]
             BMI  하루30분이상운동     운동빈도 성별 학교급
29623  15.238947        3.0  주에 3~4회  남   중
29624  20.342891        1.0  

/var/folders/qz/d378mfsj25g7bb288v8ls9yw0000gn/T/ipykernel_5722/1123144500.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data_exercise_3times['주 3회 운동여부'] = cleaned_data_exercise_3times['주3회이상운동'].map(exercise_mapping_3times)
/var/folders/qz/d378mfsj25g7bb288v8ls9yw0000gn/T/ipykernel_5722/1123144500.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data_exercise_30min['운동빈도'] = cleaned_data_exercise_30min['하루30분이상운동'].map(exercise_mapping_30min)


### 정규성 가정 (Normality Assumption): KS Test 사용

**Kolmogorov-Smirnov Test (KS Test)**는 데이터의 분포가 특정 분포(예: 정규 분포)와 얼마나 일치하는지를 확인하기 위해 사용되었습니다.

Shapiro-Wilk Test는 정규성 검정에 더 민감하지만, 큰 샘플에서 p-value의 정확도가 떨어질 수 있다는 한계가 있습니다.

따라서, 본 분석에서는 큰 샘플 데이터를 다룰 때 유리한 KS Test를 선택하여 데이터의 정규성을 판단하고, 비모수적 검정(Kruskal-Wallis Test) 필요 여부를 결정하였습니다.

In [44]:
# '주3회이상운동' 매핑 : 초등학생 대상 정규성 검정

from scipy.stats import ks_1samp, norm
import numpy as np

# KS 테스트를 위한 데이터 준비
print("\n[Kolmogorov-Smirnov Test 결과: BMI 데이터]")

for exercise_status in ['예', '아니오']:
    for gender in ['남', '여']:
        subset = cleaned_data_exercise_3times[
            (cleaned_data_exercise_3times['주 3회 운동여부'] == exercise_status) &
            (cleaned_data_exercise_3times['성별'] == gender)
        ]['BMI']

        if len(subset) > 1:  # 데이터 크기 확인
            # 평균과 표준편차를 사용하여 정규분포 생성
            mean = subset.mean()
            std = subset.std()
            standardized_data = (subset - mean) / std  # 데이터 표준화
            
            # KS 테스트 수행
            ks_stat, p_value = ks_1samp(standardized_data, cdf=norm.cdf)
            print(f"\n{gender} - {exercise_status} 그룹")
            print(f"Kolmogorov-Smirnov Test: KS Statistic={ks_stat:.4f}, p-value={p_value:.4f}")
        else:
            print(f"\n{gender} - {exercise_status} 그룹: 데이터가 부족하여 KS 테스트를 수행할 수 없습니다.")



[Kolmogorov-Smirnov Test 결과: BMI 데이터]

남 - 예 그룹
Kolmogorov-Smirnov Test: KS Statistic=0.0897, p-value=0.0000

여 - 예 그룹
Kolmogorov-Smirnov Test: KS Statistic=0.0863, p-value=0.0000

남 - 아니오 그룹
Kolmogorov-Smirnov Test: KS Statistic=0.0903, p-value=0.0000

여 - 아니오 그룹
Kolmogorov-Smirnov Test: KS Statistic=0.0849, p-value=0.0000


In [45]:
# '하루30분이상운동' 정규성 검정 
from scipy.stats import ks_1samp, norm

# KS 테스트 수행
print("\n[Kolmogorov-Smirnov Test 결과: 하루30분이상운동에 따른 BMI 데이터]")

for exercise_level in cleaned_data_exercise_30min['운동빈도'].unique():
    for gender in cleaned_data_exercise_30min['성별'].unique():
        for school_level in cleaned_data_exercise_30min['학교급'].unique():
            # 데이터 필터링
            subset = cleaned_data_exercise_30min[
                (cleaned_data_exercise_30min['운동빈도'] == exercise_level) & 
                (cleaned_data_exercise_30min['성별'] == gender) & 
                (cleaned_data_exercise_30min['학교급'] == school_level)
            ]['BMI']
            
            print(f"\n{exercise_level} - {gender} - {school_level} 그룹 데이터 크기: {len(subset)}")  # 데이터 크기 확인
            
            if len(subset) > 1:  # 데이터 크기 확인
                # 평균과 표준편차를 사용하여 정규분포 생성
                mean = subset.mean()
                std = subset.std()
                standardized_data = (subset - mean) / std  # 데이터 표준화
                
                # KS 테스트 수행
                ks_stat, p_value = ks_1samp(standardized_data, cdf=norm.cdf)
                print(f"Kolmogorov-Smirnov Test: KS Statistic={ks_stat:.4f}, p-value={p_value:.4f}")
            else:
                print("데이터가 부족하여 KS 테스트를 수행할 수 없습니다.")



[Kolmogorov-Smirnov Test 결과: 하루30분이상운동에 따른 BMI 데이터]

주에 3~4회 - 남 - 중 그룹 데이터 크기: 3081
Kolmogorov-Smirnov Test: KS Statistic=0.0661, p-value=0.0000

주에 3~4회 - 남 - 고 그룹 데이터 크기: 2920
Kolmogorov-Smirnov Test: KS Statistic=0.0533, p-value=0.0000

주에 3~4회 - 여 - 중 그룹 데이터 크기: 2058
Kolmogorov-Smirnov Test: KS Statistic=0.0697, p-value=0.0000

주에 3~4회 - 여 - 고 그룹 데이터 크기: 1265
Kolmogorov-Smirnov Test: KS Statistic=0.0643, p-value=0.0001

거의 안함 - 남 - 중 그룹 데이터 크기: 2410
Kolmogorov-Smirnov Test: KS Statistic=0.0750, p-value=0.0000

거의 안함 - 남 - 고 그룹 데이터 크기: 3566
Kolmogorov-Smirnov Test: KS Statistic=0.0525, p-value=0.0000

거의 안함 - 여 - 중 그룹 데이터 크기: 4009
Kolmogorov-Smirnov Test: KS Statistic=0.0764, p-value=0.0000

거의 안함 - 여 - 고 그룹 데이터 크기: 6904
Kolmogorov-Smirnov Test: KS Statistic=0.0808, p-value=0.0000

주에 1~2회 - 남 - 중 그룹 데이터 크기: 3253
Kolmogorov-Smirnov Test: KS Statistic=0.0683, p-value=0.0000

주에 1~2회 - 남 - 고 그룹 데이터 크기: 4490
Kolmogorov-Smirnov Test: KS Statistic=0.0550, p-value=0.0000

주에 1~2회 - 여 - 

모든 그룹에서 p-value가 0.0000으로 나타났습니다.
이는 BMI 분포가 정규성을 따르지 않는다는 것을 의미합니다.
따라서, 남학생 데이터는 정규성 가정을 만족하지 않으므로 비모수적 검정이 필요합니다

여학생 데이터에서도 모든 그룹에서 p-value가 0.0000으로 나타났습니다.
따라서, 여학생 데이터 역시 BMI 분포가 정규성을 따르지 않으므로 비모수적 검정이 필요합니다.

### 등분산성 가정 (Homogeneity of Variance) : Levene’s Test

Levene’s Test는 여러 그룹 간의 분산이 동일한지(등분산성)를 검정하는 통계적 방법입니다. 데이터가 정규성을 따르지 않더라도 사용할 수 있기 때문에, 정규성 여부와 관계없이 등분산성을 확인할 때 적합합니다.

Levene’s Test의 결과 해석
- 귀무가설 (H₀): 그룹 간의 분산이 동일하다.
- 대립가설 (H₁): 그룹 간의 분산이 동일하지 않다.
- p-value < 0.05: 귀무가설 기각 → 등분산성이 충족되지 않음.
- p-value ≥ 0.05: 귀무가설 채택 → 등분산성이 충족됨.

In [46]:
from scipy.stats import levene

# Levene's Test 수행
## '주3회이상운동' 운동 여부와 성별에 따른 등분산성 검정 

print("\n[Levene's Test 결과: 주 3회 운동 여부와 성별에 따른 BMI 데이터]")

# 그룹별 데이터 필터링
group_male_yes = cleaned_data_exercise_3times[
    (cleaned_data_exercise_3times['주 3회 운동여부'] == '예') &
    (cleaned_data_exercise_3times['성별'] == '남')
]['BMI']

group_male_no = cleaned_data_exercise_3times[
    (cleaned_data_exercise_3times['주 3회 운동여부'] == '아니오') &
    (cleaned_data_exercise_3times['성별'] == '남')
]['BMI']

group_female_yes = cleaned_data_exercise_3times[
    (cleaned_data_exercise_3times['주 3회 운동여부'] == '예') &
    (cleaned_data_exercise_3times['성별'] == '여')
]['BMI']

group_female_no = cleaned_data_exercise_3times[
    (cleaned_data_exercise_3times['주 3회 운동여부'] == '아니오') &
    (cleaned_data_exercise_3times['성별'] == '여')
]['BMI']

# 데이터 크기 확인
group_sizes = {
    "남자-예": len(group_male_yes),
    "남자-아니오": len(group_male_no),
    "여자-예": len(group_female_yes),
    "여자-아니오": len(group_female_no)
}
print("\n그룹별 데이터 크기:")
print(group_sizes)

# Levene's Test 수행
if all(size > 1 for size in group_sizes.values()):  # 모든 그룹에 데이터가 2개 이상 있어야 수행 가능
    levene_stat, p_value = levene(group_male_yes, group_male_no, group_female_yes, group_female_no)
    print("\n[Levene's Test 결과]")
    print(f"Levene's Test Statistic: {levene_stat:.4f}, p-value: {p_value:.4f}")
    
    if p_value < 0.05:
        print("귀무가설 기각: 분산이 동일하지 않음.")
    else:
        print("귀무가설 채택: 분산이 동일함.")
else:
    print("\n데이터가 부족하여 Levene's Test를 수행할 수 없습니다.")



[Levene's Test 결과: 주 3회 운동 여부와 성별에 따른 BMI 데이터]

그룹별 데이터 크기:
{'남자-예': 12238, '남자-아니오': 4515, '여자-예': 8998, '여자-아니오': 6964}

[Levene's Test 결과]
Levene's Test Statistic: 135.8066, p-value: 0.0000
귀무가설 기각: 분산이 동일하지 않음.


In [47]:
# '하루30분이상운동' 등분산성 검정 
from scipy.stats import levene

print("\n[Levene's Test 결과: 하루 30분 이상 운동 빈도와 성별, 학교급에 따른 BMI 데이터]")

grouped_data = []
for exercise_level in cleaned_data_exercise_30min['운동빈도'].unique():
    for school_level in cleaned_data_exercise_30min['학교급'].unique():
        for gender in ['남', '여']:
            group = cleaned_data_exercise_30min[
                (cleaned_data_exercise_30min['운동빈도'] == exercise_level) & 
                (cleaned_data_exercise_30min['학교급'] == school_level) & 
                (cleaned_data_exercise_30min['성별'] == gender)
            ]['BMI']
            
            if len(group) > 1:  # 데이터가 충분한 경우에만 추가
                grouped_data.append(group)

# Levene's Test 수행
if len(grouped_data) > 1:  # 적어도 두 그룹이 필요
    levene_stat, p_value = levene(*grouped_data)
    print("\n[전체 그룹 Levene's Test 결과]")
    print(f"Levene's Test Statistic: {levene_stat:.4f}, p-value: {p_value:.4f}")
    
    if p_value < 0.05:
        print("귀무가설 기각: 전체 그룹 간 분산이 동일하지 않음.")
    else:
        print("귀무가설 채택: 전체 그룹 간 분산이 동일함.")
else:
    print("데이터가 부족하여 Levene's Test를 수행할 수 없습니다.")




[Levene's Test 결과: 하루 30분 이상 운동 빈도와 성별, 학교급에 따른 BMI 데이터]

[전체 그룹 Levene's Test 결과]
Levene's Test Statistic: 54.8401, p-value: 0.0000
귀무가설 기각: 전체 그룹 간 분산이 동일하지 않음.


In [48]:
# 정규성과 등분산성모두 불만족 -> 비모수 검정 (Kruskal-Wallis Test)을 사용

# Kruskal-Wallis : 성별 그룹별로 범주형 변수인 운동 여부 (예, 아니오) 연속형 변수인 BMI 사이의 차이를 비교하고자 사용함


In [51]:
from scipy.stats import kruskal
import scikit_posthocs as sp

# Kruskal-Wallis Test 수행
print("\n[Kruskal-Wallis Test 결과: 주 3회 운동 여부와 성별에 따른 BMI 데이터]")

# 그룹별 데이터 필터링
group_male_yes = cleaned_data_exercise_3times[
    (cleaned_data_exercise_3times['주 3회 운동여부'] == '예') &
    (cleaned_data_exercise_3times['성별'] == '남')
]['BMI']

group_male_no = cleaned_data_exercise_3times[
    (cleaned_data_exercise_3times['주 3회 운동여부'] == '아니오') &
    (cleaned_data_exercise_3times['성별'] == '남')
]['BMI']

group_female_yes = cleaned_data_exercise_3times[
    (cleaned_data_exercise_3times['주 3회 운동여부'] == '예') &
    (cleaned_data_exercise_3times['성별'] == '여')
]['BMI']

group_female_no = cleaned_data_exercise_3times[
    (cleaned_data_exercise_3times['주 3회 운동여부'] == '아니오') &
    (cleaned_data_exercise_3times['성별'] == '여')
]['BMI']

# Kruskal-Wallis Test 수행
data = [
    group_male_yes,
    group_male_no,
    group_female_yes,
    group_female_no
]

if all(len(group) > 1 for group in data):  # 모든 그룹에 데이터가 2개 이상 있어야 수행 가능
    kruskal_stat, p_value = kruskal(*data)
    print("\n[Kruskal-Wallis Test 결과]")
    print(f"Kruskal-Wallis Statistic: {kruskal_stat:.4f}, p-value: {p_value:.4f}")
    
    if p_value < 0.05:
        print("귀무가설 기각: 그룹 간 BMI의 차이가 유의미함.")
        
        # Dunn's Test 수행
        print("\n[Dunn's Test 결과]")
        all_data = cleaned_data_exercise_3times[['BMI', '주 3회 운동여부', '성별']].copy()
        all_data['Group'] = (
            all_data['성별'] + "-" + all_data['주 3회 운동여부']
        )
        
        # Dunn's Test
        dunn_result = sp.posthoc_dunn(all_data, val_col='BMI', group_col='Group', p_adjust='bonferroni')
        print(dunn_result)
        
    else:
        print("귀무가설 채택: 그룹 간 BMI의 차이가 유의미하지 않음.")
else:
    print("\n데이터가 부족하여 Kruskal-Wallis Test를 수행할 수 없습니다.")



[Kruskal-Wallis Test 결과: 주 3회 운동 여부와 성별에 따른 BMI 데이터]

[Kruskal-Wallis Test 결과]
Kruskal-Wallis Statistic: 540.9976, p-value: 0.0000
귀무가설 기각: 그룹 간 BMI의 차이가 유의미함.

[Dunn's Test 결과]
              남-아니오           남-예         여-아니오           여-예
남-아니오  1.000000e+00  2.015645e-02  7.275510e-55  1.016937e-54
남-예    2.015645e-02  1.000000e+00  4.131822e-61  1.566214e-63
여-아니오  7.275510e-55  4.131822e-61  1.000000e+00  1.000000e+00
여-예    1.016937e-54  1.566214e-63  1.000000e+00  1.000000e+00


# 정규성과 등분산성모두 불만족 -> 비모수 검정 (Kruskal-Wallis Test)을 사용

---

# Kruskal-Wallis : 학교급, 성별 그룹별로 범주형 변수인 운동 빈도 (1,2,3,4) 연속형 변수인 BMI 사이의 차이를 비교하고자 사용함

In [54]:
# Kruskal-Wallis Test 수행
from scipy.stats import kruskal

print("\n[Kruskal-Wallis Test 결과: 학교급, 성별, 운동 빈도에 따른 BMI 데이터]")

# 그룹화된 데이터 준비
grouped_data = []
labels = []  # 그룹 레이블 저장
for exercise_level in cleaned_data_exercise_30min['운동빈도'].unique():
    for school_level in cleaned_data_exercise_30min['학교급'].unique():
        for gender in ['남', '여']:
            group = cleaned_data_exercise_30min[
                (cleaned_data_exercise_30min['운동빈도'] == exercise_level) & 
                (cleaned_data_exercise_30min['학교급'] == school_level) & 
                (cleaned_data_exercise_30min['성별'] == gender)
            ]['BMI']
            
            if len(group) > 1:  # 데이터가 충분한 경우에만 추가
                grouped_data.append(group)
                labels.append(f"{exercise_level}-{school_level}-{gender}")

# Kruskal-Wallis Test 수행
if len(grouped_data) > 1:  # 적어도 두 그룹이 필요
    kruskal_stat, p_value = kruskal(*grouped_data)
    print("\n[전체 그룹 Kruskal-Wallis Test 결과]")
    print(f"Kruskal-Wallis Statistic: {kruskal_stat:.4f}, p-value: {p_value:.4f}")
    
    if p_value < 0.05:
        print("귀무가설 기각: 전체 그룹 간 BMI 차이가 유의미함.")
        print("\n[사후 검정 (Dunn's Test) 수행]")

        # Dunn's Test 수행
        import scikit_posthocs as sp
        all_data = cleaned_data_exercise_30min.copy()
        all_data['Group'] = (
            all_data['운동빈도'] + "-" + 
            all_data['학교급'] + "-" + 
            all_data['성별']
        )
        
        # Dunn's Test
        dunn_result = sp.posthoc_dunn(all_data, val_col='BMI', group_col='Group', p_adjust='bonferroni')
        print("\n[Dunn's Test 결과]")
        print(dunn_result)
    else:
        print("귀무가설 채택: 전체 그룹 간 BMI 차이가 유의미하지 않음.")
else:
    print("데이터가 부족하여 Kruskal-Wallis Test를 수행할 수 없습니다.")



[Kruskal-Wallis Test 결과: 학교급, 성별, 운동 빈도에 따른 BMI 데이터]

[전체 그룹 Kruskal-Wallis Test 결과]
Kruskal-Wallis Statistic: 2256.8592, p-value: 0.0000
귀무가설 기각: 전체 그룹 간 BMI 차이가 유의미함.

[사후 검정 (Dunn's Test) 수행]

[Dunn's Test 결과]
                  거의 안함-고-남      거의 안함-고-여     거의 안함-중-남      거의 안함-중-여  \
거의 안함-고-남      1.000000e+00   1.717288e-56  4.296424e-23  1.546619e-137   
거의 안함-고-여      1.717288e-56   1.000000e+00  1.000000e+00   3.070344e-33   
거의 안함-중-남      4.296424e-23   1.000000e+00  1.000000e+00   2.303541e-30   
거의 안함-중-여     1.546619e-137   3.070344e-33  2.303541e-30   1.000000e+00   
주에 1~2회-고-남    4.755404e-02  1.828290e-100  2.828055e-42  1.237056e-199   
주에 1~2회-고-여    1.531403e-07   5.085815e-23  5.255676e-06   2.312369e-91   
주에 1~2회-중-남    2.711970e-14   3.813858e-08  6.818502e-01   2.870327e-56   
주에 1~2회-중-여    2.496786e-81   2.856456e-06  4.848473e-09   9.702591e-08   
주에 3~4회-고-남    4.674524e-05   4.194262e-94  9.011776e-46  7.013943e-183   
주에 3~4회-고-여    1.000000e+00   5.7902